<a href="https://www.kaggle.com/code/avtnshm/cricket-world-cup-2023-analysis?scriptVersionId=148134342" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
#Loading the file
import pandas as pd
import warnings
warnings.simplefilter("ignore")
cwc2023 = pd.read_csv('/kaggle/input/cwc2023-match-statistics-updated-daily/CWC2023.csv') #copy the path of the file from the sidebar.
cwc2023.tail()

,Match_No,Team_1,Team_2,Venue,Toss,Choice,Innings1_Run,Innings1_Balls,Innings1_Wickets,Innings2_Run,Innings2_Balls,Innings2_Wickets,Winner,Margin_Runs_or_Wickets
20,21,India,New Zealand,Dharamshala,India,Ball,273,300,10,274,288,6,India,4W
21,22,Afganistan,Pakistan,Chennai,Pakistan,Bat,282,300,7,286,294,2,Afganistan,8W
22,23,South Africa,Bangladesh,Mumbai,South Africa,Bat,382,300,5,233,280,10,South Africa,149R
23,24,Australia,Netherlands,New Delhi,Australia,Bat,399,300,8,90,126,10,Australia,309R
24,25,Sri Lanka,England,Bengaluru,England,Bat,156,200,10,160,154,2,Sri Lanka,8W


In [2]:
#the dataset is being updated daily till 19th November 2023, so just explore and gain insights and if you are a cricket fan, do enjoy the mathces too

In [3]:
from scipy.stats import chi2_contingency
contingency_table_toss = pd.crosstab(cwc2023['Toss'], cwc2023['Winner'])
chi2, p, _, _ = chi2_contingency(contingency_table_toss)
alpha = 0.05
if p < alpha:
    print("There is a statistically significant association between 'Toss' and 'Winner'.")
else:
    print("There is no statistically significant association between 'Toss' and 'Winner'.")



There is no statistically significant association between 'Toss' and 'Winner'.


In [4]:
contingency_table_toss

Winner,Afganistan,Australia,Bangladesh,England,India,Netherlands,New Zealand,Pakistan,South Africa,Sri Lanka
Toss,,,,,,,,,,
Afganistan,0,0,0,0,1,0,1,0,0,0
Australia,0,1,0,0,1,0,0,0,1,0
Bangladesh,0,0,1,1,1,0,0,0,0,0
England,1,0,0,0,0,0,0,0,1,1
India,0,0,0,0,2,0,0,0,0,0
Netherlands,0,0,0,0,0,0,1,1,0,1
New Zealand,0,0,0,0,0,0,2,0,0,0
Pakistan,1,1,0,0,0,0,0,0,0,0
South Africa,0,0,0,0,0,1,0,0,1,0


In [5]:
from scipy.stats import chi2_contingency
contingency_table_choice = pd.crosstab(cwc2023['Choice'], cwc2023['Winner'])
chi2, p, _, _ = chi2_contingency(contingency_table_choice)
alpha = 0.05
if p < alpha:
    print("There is a statistically significant association between 'Choice' and 'Winner'.")
else:
    print("There is no statistically significant association between 'Choice' and 'Winner'.")



There is no statistically significant association between 'Choice' and 'Winner'.


In [6]:
cwc2023['Margin'] = cwc2023.apply(lambda row: f"{row['Innings1_Run'] - row['Innings2_Run']}R" if row['Innings1_Run'] > row['Innings2_Run'] else f"{10 - row['Innings2_Wickets']}W", axis=1)
cwc2023 #adding the margin columns by pandas calculation

,Match_No,Team_1,Team_2,Venue,Toss,Choice,Innings1_Run,Innings1_Balls,Innings1_Wickets,Innings2_Run,Innings2_Balls,Innings2_Wickets,Winner,Margin_Runs_or_Wickets,Margin
0,1,New Zealand,England,Ahemedabad,New Zealand,Ball,282,300,9,283,218,1,New Zealand,9W,9W
1,2,Pakistan,Netherlands,Hyderabad,Netherlands,Ball,286,294,10,205,246,10,Pakistan,81R,81R
2,3,Bangladesh,Afganistan,Dharamshala,Bangladesh,Ball,156,224,10,158,208,4,Bangladesh,6W,6W
3,4,South Africa,Sri Lanka,New Delhi,SriLanka,Ball,428,300,5,326,269,10,South Africa,102R,102R
4,5,India,Australia,Chennai,Australia,Bat,199,297,10,201,248,4,India,6W,6W
5,6,New Zealand,Netherlands,Hyderabad,Netherlands,Ball,322,300,7,223,279,10,New Zealand,99R,99R
6,7,England,Bangladesh,Dharamshala,Bangladesh,Ball,364,300,9,227,290,10,England,137R,137R
7,8,Pakistan,Sri Lanka,Hyderabad,SriLanka,Bat,334,300,9,345,290,4,Pakistan,6W,6W
8,9,India,Afganistan,New Delhi,Afganistan,Bat,272,300,8,273,210,2,India,8W,8W
9,10,South Africa,Australia,Lucknow,Australia,Ball,311,300,7,177,345,10,South Africa,134R,134R


In [7]:
team_wins = cwc2023['Winner'].value_counts()
teams_with_3_or_more_wins = team_wins[team_wins >=3].index
matches_of_interest = cwc2023[cwc2023['Winner'].isin(teams_with_3_or_more_wins)]

In [8]:
import plotly.express as px
import geopandas as gpd
# Calculate the count of wins for each country
country_wins = matches_of_interest['Winner'].value_counts().reset_index()
country_wins.columns = ['name', 'Wins']

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
merged_data = world.merge(country_wins, on='name', how='left')

# Create a choropleth map
fig = px.choropleth(
    merged_data,
    locations="iso_a3",
    color="Wins",
    hover_name="name",
    title="Countries with 3 or more Wins in Cricket World Cup 2023",
)
print(country_wins)
fig.show()


           name  Wins
0         India     5
1   New Zealand     4
2  South Africa     4
3     Australia     3
